# Sequences selected by divergent exhibit large conventional genetic distance

We test the expectation that sequences selected in the divergent set have a greater genetic distance than a randoml selection of the sdame number of sequences.

If DNA sequences are homologous, then `divergent` should select sets of sequences within for which their minimum genetic distance is near the upper tail of the distribution of this measure from a sampling of combinations of the same (or different size).

The data was from alignments of 106 genes from exactly 31 diverse mammal species. The matrix of all pairwise genetic distances were computed between the aligned sequences using the Paralinear distance of Lake. If the JSD measure reflects genetic distance, we expect the mean of the pairwise distances between the species identified by `divergent max` to be larger than the same number of species selected randomly. We set a minimal set size of 5 and maximal set size of 10.

The column labelled "P-value<0.1" identifies how many of `num` genes for which the divergent set species gave a p-value ≤0.1 (this value was arbitrarily chosen). The distribution for each gene was obtained be taking of the mean of 1000 randomly chosen (without replacement) combinations of species.

The relationship between the statistic chosen, k and whether a post-process pruning was done all had an effect. For this data set, the combination `mean_delta_jsd`, `max_set=False` and `k=4` produced the largest relationship with genetic distance.

In [1]:
import project_path
import plotly.express as px
from cogent3 import load_table

table = load_table("jsd_v_dist-stats.tsv")
table.columns["fixed_size"] = table.columns["min_size"] == table.columns["max_size"]

## The value of $k$ impacts on the dispersal of the selected sequences

The performance of divergent `max` in recovering representative sequences is a function of $k$ and the selected statistic.

In [2]:
fig = px.scatter(
    table,
    x="k",
    y="(p-value<0.05)%",
    color="stat",
    trendline="lowess",
    facet_row="fixed_size",
    labels={
        "k": "<i>k</i>",
        "(p-value<0.05)%": "Significant %",
        "fixed_size": "Fixed Set Size",
    },
)
d_jsd = "δ<sub>JSD</sub>"
stat_names = dict(stdev=f"std({d_jsd})", cov=f"cov({d_jsd})")
tickfont = dict(size=16)
titlefont = dict(size=20)
legend = dict(title=dict(text=""), font=dict(size=17), tracegroupgap=10)
fig.update_layout(
    height=1200,
    width=1400,
    legend=legend,
    xaxis=dict(tickfont=tickfont, titlefont=titlefont),
)
fig.for_each_yaxis(lambda yaxis: yaxis.update(tickfont=tickfont, titlefont=titlefont))
fig.for_each_trace(lambda t: t.update(name=stat_names.get(t.name, t.name)))
fig.update_traces(marker=dict(size=12, opacity=0.8))
fig.update_annotations(font=titlefont)
outpath = project_path.FIG_DIR / "jsd_v_dist-stats.png"
fig.write_image(outpath)

## The distribution of the number of selected sequences

Both $k$ and the statistic impact on the number of sequences selected by divergent `max`.

In [3]:
from plotly.graph_objects import Scatter, Figure
from cogent3 import load_table, make_table


def make_plot(table):
    # this has to be done manually because plotly express has a
    # bug with error bars and multiple traces
    tickfont = dict(size=16)
    titlefont = dict(size=20)
    legend = dict(title=dict(text=""), font=dict(size=17), tracegroupgap=10)
    stdev = table.filtered(lambda x: x == "stdev", columns="stat")
    cov = table.filtered(lambda x: x == "cov", columns="stat")
    fig = Figure()
    st = Scatter(
        x=stdev.columns["k"],
        y=stdev.columns["mean_size"],
        mode="markers",
        name="stdev",
        error_y=dict(
            type="data",
            array=stdev.columns["error_above"].tolist(),
            arrayminus=stdev.columns["error_below"].tolist(),
        ),
    )
    fig.add_trace(st)
    cv = Scatter(
        x=cov.columns["k"],
        y=cov.columns["mean_size"],
        mode="markers",
        name="cov",
        error_y=dict(
            type="data",
            array=cov.columns["error_above"].tolist(),
            arrayminus=cov.columns["error_below"].tolist(),
        ),
    )
    fig.add_trace(cv)
    fig.update_layout(
        height=600,
        width=1400,
        xaxis=dict(
            title="<i>k</i>",
            titlefont=titlefont,
            tickfont=tickfont,
        ),
        yaxis=dict(
            title="Mean Size",
            titlefont=titlefont,
            tickfont=tickfont,
        ),
        legend=legend,
    )
    fig.update_traces(marker=dict(size=12))
    fig.update_annotations(font=titlefont)

    return fig


def calc_upper_lower_bounds(x, y, lower, upper):
    assert lower <= x <= upper, f"invalid {x=}, {lower=}, {upper=}"
    lo = x - y
    hi = x + y
    if lo >= lower and hi <= upper:
        return y, y
    elif lo <= lower and hi >= upper:
        return x - lower, upper - x
    elif lo < lower:
        diff = x - lower
        return x - lower, y - diff
    else:
        diff = upper - x
        return (y - diff), upper - x


def group_by(table, one_stat):
    columns = ["k", "min_size", "max_size", "stat"]
    distinct = table.distinct_values(columns)

    results = []
    for group in distinct:
        _, lower, upper, _ = group
        subtable = table.filtered(lambda x: tuple(x) == group, columns=columns)
        sizes = subtable.columns["size"]
        mean, std = sizes.mean(), sizes.std(ddof=1)
        lo, hi = calc_upper_lower_bounds(mean, std, lower, upper)
        results.append(list(group) + [mean, std, lo, hi, sizes.shape[0]])

    table = make_table(
        header=columns + ["mean_size", "std_size", "error_below", "error_above", "n"],
        rows=results,
    )
    table = table.sorted(columns=("stat", "k"))
    if one_stat:
        table = table.filtered(lambda x: x == "stdev", columns="stat")
    return table


table = load_table(
    "/Users/gavin/repos/Divergent/paper/nbks/jsd_v_dist-sizes.tsv"
).filtered(lambda x: x == 5, columns="min_size")
table = group_by(table, one_stat=False)
fig = make_plot(table)
outpath = project_path.FIG_DIR / "jsd_v_dist-sizes.png"
fig.write_image(outpath)